<a href="https://colab.research.google.com/github/Shuregg/miet-analog-electronics/blob/main/DFV_LAB1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# @title Датчики Физических Величин | Lab 1 Моделирование интегрального тензомоста

import numpy as np

gamma = 0.22 * 10**(-2)         # Температурный коэффициент сопротивления (ТКС) резистора R0 моста [1/C]
alpha = -0.9 * 10**(-2)         # ТКС компенсационного резистора [1/C]
alpha_t = -1.0 * 10**(-2)         # ТКС компенсац. резистора для термокомпенсированного моста
dT = 45                         # [degrees Cels]
R0 = 1.2 * 1000                 # [Ohm]
x_max = 0.35 * 10**(-2)         # +- Xmax
g = 0.9 * 1000                  # Чувствительность управляемого резистора [Ом/В]/[Ohm/V]
E = 9                           # источник ЭДС [V]
Rk = 1.1 * 1000                 # Ohm

eps_T = gamma * dT

K = 10                          # Op Amp coef

def beta_p(x, R_T0):
  beta_p = -(E * x / K) * ( ((R_T0 / R0) * alpha + gamma) / (1 + (R_T0 / R0) * (1 + alpha * dT) + gamma * dT)**2 )
  return beta_p

def beta_aver(dU1, dU2, dT, K):
  return (dU1 - dU2) / (2*dT*K)

def S_T(U_out0, x, K):
  return U_out0 / (x * K)
# R1 = R0 * (1 + eps_x + eps_T)
# R2 = R0 * (1 - eps_x + eps_T)
# R3 = R0 * (1 - eps_x + eps_T)
# R4 = R0 * (1 + eps_x + eps_T)
# R5
# R6
# Rfb

a1 = 1    # a1 = Rfb / R1
a2 = 1    # a2 = Rfb / R2
a3 = 1    # a3 = Rfb / R3
a4 = 1    # a4 = R6 / R4
a5 = 0.25 # a5 = R6 / R5

# 1. Устанавливаем номинальные сопротивления плеч моста R0.
# (Рассчитываем U0 по заданному R0, g и произвольно выбранному a1 = 1)
U0 = - R0 / (g * a1)  # Для установки начального значения сопротивления плеча моста R0
                      # (при нулевых значениях Uд, Uд, Ec(T) ) используется напряжение U0.

# 2. Вычисляем амплитуду вxодного воздействия UД по рассчитанному U0 и выбранным
# значениям a1 = 1, a2 = 1.
Ud = x_max * U0 * (a2 / a1) # Дифференциальная составляющая UД(x) (+/-)

# 3. Вычисляем синфазную (температурную) составляющую в сигнале управления.
Ec = (gamma * R0 * dT) / (a4 * g) # Синфазная составляющая

# 4. Вычисляем номинальное сопротивление терморезистора при заданном alpha = -1% * C^1.
# Условие (R_c0 = (gamma / alpha) * R0) имеет физический смысл, если alpha и gamma имеют противоположные знаки.
# Например для кремниевого тензомоста, где резисторы имеют положительный ТКС,
# в качестве термокомпенсатора подходят терморезисторы, имеющие отрицательный ТКС.
R_T0 = - (gamma * R0) / alpha_t

# 5. Устанавливаем номинальное сопротивление терморезистора R_T0
#  (вычисляемнапряжение смещения Uсм )
Uoffset = R_T0 / (g * a5)

dU1 = [-28.4 , 28.3, -8.8, 8.8, -5.9, 6, -0.2, 0.1]
dU2 = [34.5, -34.5, 10.1, -10.1, 6.5, -6.5, 0.1, -0.1]

# dU1 = [-28.4 * 10 ** -3, 28.3 * 10 ** -3, -8.8* 10 ** -3, 8.8* 10 ** -3, -5.9* 10 ** -3, 6* 10 ** -3, -0.2* 10 ** -3, 0.1* 10 ** -3]
# dU2 = [34.5* 10 ** -3, -34.5* 10 ** -3, 10.1* 10 ** -3, -10.1* 10 ** -3, 6.5* 10 ** -3, -6.5* 10 ** -3, 0.1* 10 ** -3, -0.1* 10 ** -3]

print("Initial data")
print("γ: ", gamma*10**(2), "%, [1/C]")
print("α: ", np.round(alpha*10**(2), 1), "%, [1/C]")
print("ΔT: ", dT, "[ºC]")
print("R0: ", R0, "[Ohm]")
print("±x_max: ±  ", x_max*10**(2), "%")
print("g: ", g, "[Ohm/V]")
print("E: ", E, "[V]")
print("Rк: ", Rk, "[Ohm]")

print("\nOutput data")
print("U0: ", np.round(U0, 4), "[V]")
print("Uд: ", np.round(Ud, 5), "[V]")
print("Ec: ", Ec, "[V]")
print("R_T0: ", R_T0, "[Ohm]")
print("Uсмещ: ", np.round(Uoffset, 4), "[V]")
print()
for i in range (len(dU1)):
  print("Beta average (dU1 =", dU1[i], "| dU2 =", dU2[i], ") =", np.round(beta_aver(dU1[i], dU2[i], dT, K)*10**3, 2), "[uV/C]")
print()

x_list = [+x_max, -x_max, +x_max, -x_max, +x_max, -x_max, +x_max, -x_max]
Rk_list = [0, 0, 1000, 1000, 1500, 1500, R_T0, R_T0]

for i in range(len(x_list)):
  if(x_list[i] >0):
    print("beta_p (x_max = ", np.round(x_list[i]* 100, 2) , "Rk =", Rk_list[i], ") =", np.round(beta_p(x_list[i], Rk_list[i])*10**6, 2), "[mV/C]")
  else:
    print("beta_p (x_max =", np.round(x_list[i]* 100, 2) , "Rk =", Rk_list[i], ") =", np.round(beta_p(x_list[i], Rk_list[i])*10**6, 2), "[mV/C]")

  # print("beta_p (x_max =", np.round(x_list[i]* 100, 2) , "Rk =", Rk_list[i], ") =", beta_p(x_list[i], Rk_list[i]))


# print("\nBeta_p(+x_max, без комп.): ", beta_p(x_max, 0))
# print("Beta_p(-x_max, без комп.): ", beta_p(-x_max, 0))

# print("Beta_p(+x_max, Rk = 1.0 kOhm): ", beta_p(x_max, 1000))
# print("Beta_p(-x_max, Rk = 1.0 kOhm): ", beta_p(-x_max, 1000))

# print("Beta_p(+x_max, Rk = 1.5 kOhm): ", beta_p(x_max, 1500))
# print("Beta_p(-x_max, Rk = 1.5 kOhm): ", beta_p(-x_max, 1500))

# print("Beta_p(+x_max, Rk = R_T0): ", beta_p(x_max, R_T0))
# print("Beta_p(-x_max, Rk = R_T0): ", beta_p(-x_max, R_T0))

U_out0 = [312.4 * 10**-3, -312.4 * 10**-3, 170.4 * 10**-3, -170.5 * 10**-3, 138.8 * 10**-3, -138.9 * 10**-3, 256.3 * 10**-3, -256.3 * 10**-3]
U_T = 293  * 10**-3
print()
for i in range (len(U_out0)):
  if(x_list[i] > 0):
    print("S_T (x_max = ", np.round(x_list[i]* 100, 2) , "%, U_out0 =", np.round(U_out0[i], 4), ") =", np.round(S_T(U_out0[i], x_list[i], K)*10, 2), "[mV/%]")
  else:
    print("S_T (x_max =", np.round(x_list[i]* 100, 2) , "%, U_out0 =", np.round(U_out0[i], 4), ") =", np.round(S_T(U_out0[i], x_list[i], K)*10, 2), "[mV/%]")




Initial data
γ:  0.22 %, [1/C]
α:  -0.9 %, [1/C]
ΔT:  45 [ºC]
R0:  1200.0 [Ohm]
±x_max: ±   0.35 %
g:  900.0 [Ohm/V]
E:  9 [V]
Rк:  1100.0 [Ohm]

Output data
U0:  -1.3333 [V]
Uд:  -0.00467 [V]
Ec:  0.132 [V]
R_T0:  264.0 [Ohm]
Uсмещ:  1.1733 [V]

Beta average (dU1 = -28.4 | dU2 = 34.5 ) = -69.89 [uV/C]
Beta average (dU1 = 28.3 | dU2 = -34.5 ) = 69.78 [uV/C]
Beta average (dU1 = -8.8 | dU2 = 10.1 ) = -21.0 [uV/C]
Beta average (dU1 = 8.8 | dU2 = -10.1 ) = 21.0 [uV/C]
Beta average (dU1 = -5.9 | dU2 = 6.5 ) = -13.78 [uV/C]
Beta average (dU1 = 6 | dU2 = -6.5 ) = 13.89 [uV/C]
Beta average (dU1 = -0.2 | dU2 = 0.1 ) = -0.33 [uV/C]
Beta average (dU1 = 0.1 | dU2 = -0.1 ) = 0.22 [uV/C]

beta_p (x_max =  0.35 Rk = 0 ) = -5.74 [mV/C]
beta_p (x_max = -0.35 Rk = 0 ) = 5.74 [mV/C]
beta_p (x_max =  0.35 Rk = 1000 ) = 6.56 [mV/C]
beta_p (x_max = -0.35 Rk = 1000 ) = -6.56 [mV/C]
beta_p (x_max =  0.35 Rk = 1500 ) = 8.4 [mV/C]
beta_p (x_max = -0.35 Rk = 1500 ) = -8.4 [mV/C]
beta_p (x_max =  0.35 Rk = 264.0 